# Piru Music Composer

In [ ]:
import guitarpro


## Loading data

Since GuitarPro format express the music in terms of ch 

In [68]:
with open('data/Guitar Pro Tabs #1/Banjo/Abernathy/Abernathy, Barry - You Still Call Me Baby.gp3', 'rb') as file:
    song = guitarpro.parse(file)
    track = guitarpro.Track(song)

print('Beat position expressed in 16th of measure')
for track in song.tracks:
    print('Track', track.number, track.name)
    for measure in track.measures:
        print('')
        print('----Measure', measure.number)
        for voice in measure.voices:
            for beat in voice.beats:
                print('-Beat start',int((beat.start/measure.length+1-measure.number)*16-4),'duration', int(1/beat.duration.value*16))
                for note in beat.notes:
                    print('String',note.string,'Fret', note.value)

Beat position expressed in 16th of measure
Track 1 Banjo

----Measure 1
-Beat start 0 duration 8
String 1 Fret 0
String 5 Fret 0
-Beat start 8 duration 4
String 4 Fret 0
-Beat start 12 duration 4
String 3 Fret 0

----Measure 2
-Beat start 0 duration 1
String 3 Fret 2
-Beat start 1 duration 1
String 3 Fret 3
-Beat start 2 duration 2
String 2 Fret 0
-Beat start 4 duration 2
String 1 Fret 0
-Beat start 6 duration 2
String 5 Fret 0
-Beat start 8 duration 2
String 1 Fret 0
-Beat start 10 duration 2
String 2 Fret 0
-Beat start 12 duration 1
String 3 Fret 3
-Beat start 13 duration 1
String 3 Fret 2
-Beat start 14 duration 2
String 1 Fret 0

----Measure 3
-Beat start 0 duration 4
String 3 Fret 0
-Beat start 4 duration 2
String 5 Fret 0
-Beat start 6 duration 2
String 1 Fret 0
-Beat start 8 duration 1
String 3 Fret 2
-Beat start 9 duration 1
String 3 Fret 3
-Beat start 10 duration 2
String 2 Fret 0
-Beat start 12 duration 2
String 1 Fret 0
-Beat start 14 duration 2
String 5 Fret 0

----Measure 

String 3 Fret 0
-Beat start 4 duration 8
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 12 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 18
-Beat start 0 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 4 duration 8
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 12 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 19
-Beat start 0 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 4 duration 8
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 12 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 20
-Beat start 0 duration 16
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
String 6 Fret 3
Track 3 Mandolin

----Measure 1
-Beat start 0 duration 16

----Measure 2
-Beat start 0 duration 4
-Beat start 4 duration 4
String 1 Fret 0
String 2 Fret 0
String 3 Fret 0
String 4 Fret 0
-Beat start 8 duration 4
-Beat start 12 duration 4
String 1 Fret 0
Strin